## Topographic Complexity/Variability: Fractal Dimension   
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 

### Initial setup

In [51]:
import numpy as np
import rasterio
from scipy.stats import linregress
from scipy.ndimage import generic_filter

import joblib
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

### Define data path

In [52]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
output_file_name = 'Test_pyFractalD.tif'
#output_file_name = 'Tokeland_pyFractalD.tif'
#output_file_name = 'Nemah_pyFractalD.tif'
#output_file_name = 'Francies_pyFractalD.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [53]:
# Open the input GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the first band (assumed to be elevation data)
    dem = src.read(1)
    # Retrieve the metadata from the source GeoTIFF to use for the output
    meta = src.meta

See coordinate system info of the GeoTIFF

In [54]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

CRS: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",45.3333333333333],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",45.8333333333333],PARAMETER["standard_parallel_2",47.3333333333333],PARAMETER["false_easting",1640416.66666667],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
CRS as WKT: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitud

## Fractal Dimension (D)

Fractal dimension (D) is a measure of surface complexity, where higher values indicate more complex terrain. The calculation of fractal dimension using the variogram method consists of the following steps:

1. **Variogram Calculation**: The variogram is a function that quantifies the spatial variation of the terrain by calculating the semivariance of pixel elevation values at different lags (distances). The semivariance $ \gamma(h) $ for a lag $ h $ is computed using the formula:

$$
\gamma(h) = \frac{1}{2n(h)} \sum_{i=1}^{n} \sum_{j=1}^{n} (z_i - z_j)^2
$$

where:
- $ \gamma(h) $ is the semivariance at lag $ h $,
- $ n(h) $ is the number of pixel pairs at lag $ h $,
- $ z_i $, $ z_j $ are the elevation values of the pixel pairs.

2. **Log-Log Regression**: The log-log regression is performed on the calculated variogram values. This involves plotting the log of the variogram values against the log of the lag distances and fitting a straight line to the points.

3. **Fractal Dimension Estimation**: The fractal dimension is estimated from the slope of the regression line obtained in the log-log plot. The relationship between the slope $ m $ and the fractal dimension $ D $ is given by the equation:

$$
D = 2 - \frac{m}{2}
$$

The fractal dimension thus provides a scalar value that characterizes the complexity of the terrain surface. A higher fractal dimension indicates a more complex and rough surface.

### Calculating local Fractal Dimension (D) to detect spatial variation in surface complexity

In heterogeneous landscapes, the fractal dimension can vary across the terrain. To capture this spatial variation, the fractal dimension is calculated locally for each pixel within a moving window. The local variogram method is applied, which computes the variogram for each cell based on its surrounding neighborhood defined by the window size. The log-log regression is then performed for each local window, resulting in a raster where each pixel's value represents the local fractal dimension of the surface around that pixel.

This approach allows for the assessment of spatial variations in terrain complexity, which can be crucial for applications such as habitat mapping and geomorphological analysis.


##### Functions: Before optimization (Slow)

In [ ]:
def calculate_local_variogram(window, lags):
    """
    Calculate local variogram for a moving window.

    :param window: 1D array of the moving window values.
    :param lags: Array of lag distances.
    :return: Variogram values for the given lags.
    """
    n = int(np.sqrt(window.size))
    window = window.reshape((n, n))
    center = (n - 1) // 2
    variogram = np.zeros_like(lags, dtype=float)
    
    for i, lag in enumerate(lags):
        if center - lag < 0 or center + lag >= n:
            continue
        # Extract the ring around the central cell with the specific lag
        ring = window[center - lag : center + lag + 1, center - lag : center + lag + 1]
        central_value = window[center, center]
        # Compute the semivariance
        squared_differences = (ring - central_value) ** 2
        variogram[i] = np.nanmean(squared_differences)
    
    return variogram

def calculate_fractal_dimension(window, max_lag):
    """
    Calculate the fractal dimension for the window.

    :param window: 1D array of the moving window values.
    :param max_lag: Maximum lag distance to consider for variogram calculation.
    :return: Fractal dimension of the window.
    """
    lags = np.arange(1, max_lag + 1)
    variogram = calculate_local_variogram(window, lags)
    
    # Perform log-log linear regression on the variogram values
    log_lags = np.log(lags[variogram > 0])
    log_variogram = np.log(variogram[variogram > 0])
    
    if len(log_lags) < 2:  # Not enough data to fit a regression line
        return np.nan
    
    slope, intercept, r_value, p_value, std_err = linregress(log_lags, log_variogram)
    
    # The fractal dimension is related to the slope of the regression line
    fractal_dimension = 2 - slope / 2
    return fractal_dimension

def compute_fractal_dimensions(dem, window_size):
    """
    Compute fractal dimension for each cell in the raster using a moving window.

    :param dem: 2D array of elevation values.
    :param window_size: Size of the moving window to calculate the variogram.
    :return: 2D array of fractal dimension values.
    """
    max_lag = window_size // 2
    fractal_dimension_map = generic_filter(dem, calculate_fractal_dimension, size=(window_size, window_size), extra_keywords={'max_lag': max_lag})
    return fractal_dimension_map

##### Functions: Optimized via Chunk-based processing (Better)

In [ ]:
def calculate_local_variogram(window, lags):
    """
    Calculate local variogram for a moving window.

    :param window: 1D array of the moving window values.
    :param lags: Array of lag distances.
    :return: Variogram values for the given lags.
    """
    n = int(np.sqrt(window.size))
    window = window.reshape((n, n))
    center = (n - 1) // 2
    variogram = np.zeros_like(lags, dtype=float)
    
    for i, lag in enumerate(lags):
        if center - lag < 0 or center + lag >= n:
            continue
        # Extract the ring around the central cell with the specific lag
        ring = window[center - lag : center + lag + 1, center - lag : center + lag + 1]
        central_value = window[center, center]
        # Compute the semivariance
        squared_differences = (ring - central_value) ** 2
        variogram[i] = np.nanmean(squared_differences)
    
    return variogram

def calculate_fractal_dimension(window, max_lag):
    """
    Calculate the fractal dimension for the window.

    :param window: 1D array of the moving window values.
    :param max_lag: Maximum lag distance to consider for variogram calculation.
    :return: Fractal dimension of the window.
    """
    lags = np.arange(1, max_lag + 1)
    variogram = calculate_local_variogram(window, lags)
    
    # Perform log-log linear regression on the variogram values
    log_lags = np.log(lags[variogram > 0])
    log_variogram = np.log(variogram[variogram > 0])
    
    if len(log_lags) < 2:  # Not enough data to fit a regression line
        return np.nan
    
    slope, intercept, r_value, p_value, std_err = linregress(log_lags, log_variogram)
    
    # The fractal dimension is related to the slope of the regression line
    fractal_dimension = 2 - slope / 2
    return fractal_dimension

def compute_fractal_dimensions_chunked(dem, window_size):
    max_lag = window_size // 2
    rows, cols = dem.shape
    fractal_dimension_map = np.full((rows, cols), np.nan, dtype=np.float32)

    # Calculate number of chunks in each dimension
    num_chunks_row = (rows - 2 * max_lag) // chunk_size + (1 if (rows - 2 * max_lag) % chunk_size else 0)
    num_chunks_col = (cols - 2 * max_lag) // chunk_size + (1 if (cols - 2 * max_lag) % chunk_size else 0)
    
    pbar = tqdm(total=num_chunks_row * num_chunks_col, desc='Computing Fractal Dimensions')

    for i in range(max_lag, rows - max_lag, chunk_size):
        for j in range(max_lag, cols - max_lag, chunk_size):
            # Adjust chunk size for edges
            chunk_end_i = min(i + chunk_size, rows - max_lag)
            chunk_end_j = min(j + chunk_size, cols - max_lag)
            
            chunk_slice = (slice(i - max_lag, chunk_end_i + max_lag),
                           slice(j - max_lag, chunk_end_j + max_lag))
            dem_chunk = dem[chunk_slice]

            # Calculate fractal dimension for the chunk
            fractal_chunk = generic_filter(dem_chunk, calculate_fractal_dimension,
                                           size=window_size, mode='reflect',
                                           extra_arguments=(max_lag,))
            
            # Insert calculated chunk into the fractal dimension map
            map_slice = (slice(i, chunk_end_i),
                         slice(j, chunk_end_j))
            fractal_dimension_map[map_slice] = fractal_chunk[max_lag:chunk_end_i - i + max_lag, 
                                                             max_lag:chunk_end_j - j + max_lag]
            
            pbar.update(1)
    
    pbar.close()
    return fractal_dimension_map

#### Functions: Optimized via Parallel Processing (Fastest!)

In [55]:
def calculate_local_variogram(window, lags):
    """
    Calculate local variogram for a moving window.

    :param window: 1D array of the moving window values.
    :param lags: Array of lag distances.
    :return: Variogram values for the given lags.
    """
    n = int(np.sqrt(window.size))
    window = window.reshape((n, n))
    center = (n - 1) // 2
    variogram = np.zeros_like(lags, dtype=float)
    
    for i, lag in enumerate(lags):
        if center - lag < 0 or center + lag >= n:
            continue
        # Extract the ring around the central cell with the specific lag
        ring = window[center - lag : center + lag + 1, center - lag : center + lag + 1]
        central_value = window[center, center]
        # Compute the semivariance
        squared_differences = (ring - central_value) ** 2
        variogram[i] = np.nanmean(squared_differences)
    
    return variogram

def calculate_fractal_dimension(window, max_lag):
    """
    Calculate the fractal dimension for the window.

    :param window: 1D array of the moving window values.
    :param max_lag: Maximum lag distance to consider for variogram calculation.
    :return: Fractal dimension of the window.
    """
    lags = np.arange(1, max_lag + 1)
    variogram = calculate_local_variogram(window, lags)
    
    # Perform log-log linear regression on the variogram values
    log_lags = np.log(lags[variogram > 0])
    log_variogram = np.log(variogram[variogram > 0])
    
    if len(log_lags) < 2:  # Not enough data to fit a regression line
        return np.nan
    
    slope, intercept, r_value, p_value, std_err = linregress(log_lags, log_variogram)
    
    # The fractal dimension is related to the slope of the regression line
    fractal_dimension = 2 - slope / 2
    return fractal_dimension

def process_cell(dem, i, j, max_lag):
    """
    Process a single cell to compute the fractal dimension.
    
    :param dem: 2D array of elevation values.
    :param i: Row index of the cell.
    :param j: Column index of the cell.
    :param max_lag: Maximum lag distance for variogram calculation.
    :return: Fractal dimension of the cell.
    """
    window = dem[i - max_lag:i + max_lag + 1, j - max_lag:j + max_lag + 1].flatten()
    return calculate_fractal_dimension(window, max_lag)


def compute_fractal_dimensions_parallel(dem, window_size):
    """
    Compute fractal dimensions for the raster using joblib for parallel processing.
    
    :param dem: 2D array of elevation values.
    :param window_size: Size of the moving window for variogram calculation.
    :return: 2D array of fractal dimension values.
    """
    rows, cols = dem.shape
    max_lag = window_size // 2
    indices = [(i, j) for i in range(max_lag, rows - max_lag)
                      for j in range(max_lag, cols - max_lag)]

    fractal_dimension_map = np.full(dem.shape, np.nan, dtype=np.float32)

    # Create a partial function for processing a cell
    partial_process_cell = delayed(process_cell)

    # Set up parallel processing using joblib
    num_cores = joblib.cpu_count()
    
    # Perform parallel processing, showing a progress bar
    results = Parallel(n_jobs=num_cores)(partial_process_cell(dem, i, j, max_lag) for i, j in tqdm(indices, desc='Processing'))

    # Populate the fractal_dimension_map with the results
    for (i, j), fractal_dimension in zip(indices, results):
        fractal_dimension_map[i, j] = fractal_dimension

    return fractal_dimension_map

##### Calculate D (Slow)

In [ ]:
# Define parameters
window_size = 9 #the edge length of the grid for N x N a moving window

# Calculate the fractal dimensions for the entire DEM with a specified window size
fractal_dims = compute_fractal_dimensions(dem, window_size)

##### Calculate D for the in chuncked DEM (Better)

In [ ]:
# Define parameters
window_size = 9 #the edge length of the grid for N x N a moving window

# Define the chunk size (adjust as needed)
chunk_size = 512

# Calculate the fractal dimensions for the entire DEM with a specified window size
fractal_dims = compute_fractal_dimensions_chunked(dem, window_size)

#### Calculate D - Optimized by Parallel Processing (Fastest)

In [56]:
# Define parameters
window_size = 10 #the edge length of the grid for N x N a moving window

# Calculate the fractal dimensions for the entire DEM with a specified window size
fractal_dims = compute_fractal_dimensions_parallel(dem, window_size)

Processing:   0%|          | 0/1827966 [00:00<?, ?it/s]

##### Output data into GeoTIFF  
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [57]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write the fractal dimension data to the output GeoTIFF file
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(fractal_dims.astype(rasterio.float32), 1)